设计思路：

假设从某个时间点开始，每周淘汰最后10%。


In [1]:
import pandas as pd
import math
import gc

import base
import fund_scanner.common_tools.database as db

engine = db.get_sqlalchemy_engine()

In [ ]:
# 准备所有参赛者

sql = 'select * from funds a left join funds_update b on a.funds_id=b.funds_id'

df_all_candidates = pd.read_sql(sql, engine).iloc[:,[0,1,2,3,4,5,11]]

df_all_candidates = df_all_candidates.set_index('funds_id')

df_all_candidates['last_price'] = math.nan
df_all_candidates['current_price'] = math.nan
df_all_candidates['gain_ratio'] = math.nan
df_all_candidates['good'] = 0
good = {
    'not_in_yet' : 0,
    'healthy' : 1,
    'hurt_once' : 2,
    'hurt_twice' : 3,
    'out' : 10,
    'noway' : 11
    }
# good值:
#     0 : 未参赛
#     1 : 已参赛，正活跃
#     2 : 受伤 1次
#     3 : 受伤 2次
#     10 : 淘汰
#     11 : 没有参赛资格
# last_price: 上一轮净值
# current_price: 这一轮净值
# gain_ratio: 这一轮涨幅

df_all_candidates = df_all_candidates.sort_values('funds_start_date')

df_all_candidates.tail(3)

,funds_code,funds_name_full,funds_type,funds_start_date,update_time,funds_amount,last_price,current_price,gain_ratio,good
funds_id,,,,,,,,,,
5418,001817,易方达瑞兴灵活配置混合I,None,NaT,2017-07-29 09:34:16,NaN,NaN,NaN,NaN,0
5426,001988,南方纯元A,None,NaT,2017-07-29 09:35:12,NaN,NaN,NaN,NaN,0
5439,004911,中加纯债定开债券A,None,NaT,2017-07-29 09:36:17,NaN,NaN,NaN,NaN,0


In [ ]:
gc.collect()
# 重新读取数据
df_competition = df_all_candidates.copy()

# 2017年1月1日以后成立的基金或者没有写成立时间的基金没有参赛资格
df_competition.loc[df_competition['funds_start_date']>pd.to_datetime('2013-1-1'), 'good'] = good['out']
df_competition.loc[df_competition['funds_start_date'].isnull(),'good'] = good['out']

#资产规模小于1亿，或者没有数据的没有资格参赛
df_competition.loc[df_competition['funds_amount']<1, 'good'] = good['out']
df_competition.loc[df_competition['funds_amount'].isnull(),'good'] = good['out']


#比赛从最早一直基金成立开始
start_point = pd.to_datetime(df_competition.iloc[0,3])
#比赛从2013年1月1日开始
start_point = pd.to_datetime('2013-1-1')
round_count = 1
current_date = start_point
df_competition_results = []
while current_date < pd.to_datetime('today'):
    
    # 得到当天价格（如果当天没几个价格就查次日的）
    df_competition['last_price'] = df_competition['current_price']
    while True:
        df_current_price = pd.read_sql('select * from funds_historical_price where funds_price_date=\'%s\''%current_date, engine)
        if len(df_current_price)>10:
            break
        else:
            current_date = current_date + pd.DateOffset(1)

    df_current_price = df_current_price.set_index('funds_id')
    df_competition['current_price'].update(df_current_price['funds_price_adjust'])
    
    # 如果价格从0到有，则说明参赛了，设置为 参赛1
    df_competition.loc[(df_competition['last_price']>0) & (df_competition['good']==good['not_in_yet']), 'good']=good['healthy']

    
    # 如果价格从非0到有，则说明价格变化了，计算变化率
    df_competition['gain_ratio'] = \
    ( df_competition['current_price'] - df_competition['last_price'] ) / df_competition['last_price']

    # 按变化率排序，将最后10%选手设置为 淘汰
    active_candidates = len(df_competition.loc[(df_competition['good']==good['healthy'])])
    should_be_removed = active_candidates // 10
    if should_be_removed>1:
        current_competition = df_competition.loc[(df_competition['good']==good['healthy'])].sort_values('gain_ratio')
        min_gain_ratio = current_competition['gain_ratio'].min()
        max_gain_ratio = current_competition['gain_ratio'].max()
        threshold = current_competition.iloc[should_be_removed, :]['gain_ratio']
        print('Round %d: %s: 最低涨幅 %f, 最高涨幅 %f ----> 淘汰 %d/%d 名选手，他们当期收益率低于 %f'\
              %(round_count, current_date, min_gain_ratio, max_gain_ratio, should_be_removed, active_candidates, threshold))
        df_competition.loc[(df_competition['good']==good['healthy']) & (df_competition['gain_ratio']<threshold), 'good'] = good['out']
    round_count += 1
    current_date = current_date + pd.DateOffset(4)
    df_competition_results.append(df_competition.copy())
    

# The winner is:
df_competition.loc[df_competition['good']==good['healthy']].sort_values('current_price', ascending=False)

Round 2: 2013-01-08 00:00:00: 最低涨幅 -0.019547, 最高涨幅 0.047786 ----> 淘汰 85/852 名选手，他们当期收益率低于 -0.001998
Round 3: 2013-01-14 00:00:00: 最低涨幅 0.000000, 最高涨幅 0.050696 ----> 淘汰 76/768 名选手，他们当期收益率低于 0.001974
Round 4: 2013-01-18 00:00:00: 最低涨幅 -0.005877, 最高涨幅 0.042955 ----> 淘汰 69/693 名选手，他们当期收益率低于 0.001988
Round 5: 2013-01-22 00:00:00: 最低涨幅 -0.028116, 最高涨幅 0.019113 ----> 淘汰 62/624 名选手，他们当期收益率低于 -0.012085
Round 6: 2013-01-28 00:00:00: 最低涨幅 -0.024703, 最高涨幅 0.042766 ----> 淘汰 56/562 名选手，他们当期收益率低于 0.000377
Round 7: 2013-02-01 00:00:00: 最低涨幅 -0.013193, 最高涨幅 0.049850 ----> 淘汰 50/506 名选手，他们当期收益率低于 0.000983
Round 8: 2013-02-05 00:00:00: 最低涨幅 -0.013109, 最高涨幅 0.020278 ----> 淘汰 45/457 名选手，他们当期收益率低于 -0.001963
Round 9: 2013-02-18 00:00:00: 最低涨幅 -0.018850, 最高涨幅 0.050495 ----> 淘汰 41/412 名选手，他们当期收益率低于 -0.002492
Round 10: 2013-02-22 00:00:00: 最低涨幅 -0.046776, 最高涨幅 0.005300 ----> 淘汰 37/371 名选手，他们当期收益率低于 -0.032370
Round 11: 2013-02-26 00:00:00: 最低涨幅 -0.014547, 最高涨幅 0.006472 ----> 淘汰 33/334 名选手，他们当期收益率低于 -0.006479
Rou

In [ ]:
df_competition_results[0][df_competition_results[1]['good']==good['healthy']].sort_values('gain_ratio')

In [ ]:
df_competition_results[2].sort_values('gain_ratio')